## Load Mojo Kernels

In [1]:
import os, numpy, torch
from pathlib import Path
from max.torch import CustomOpLibrary

assert torch.cuda.is_available()
op_dir = os.path.abspath('operations')

## Simple `add_one` Operation

In [32]:
op_lib = CustomOpLibrary(Path(op_dir))
add_one = op_lib.my_add_constant[{"value": 1}]

In [12]:
import time

def torch_add_one(inputs):
    return inputs + 1

def mojo_add_one(inputs):
    outputs = torch.zeros_like(inputs)
    add_one(outputs, inputs)
    return outputs

for device in ["cpu", "cuda"]:
    for op in [torch_add_one, mojo_add_one]:
        x = torch.zeros(1024, device=device)
        x = op(x) # warm-up
        start = time.perf_counter()
        for _ in range(1000):
            x = op(x)
        end = time.perf_counter()
        print(op.__name__, device, x, end - start)

torch_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.0034156449837610126
mojo_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.13251558190677315
torch_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.0074024859350174665
mojo_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.11501298600342125


## Different MatMul Algorithms

In [2]:
from max.driver import Accelerator, accelerator_count, Tensor
import torch
M, K, N = 8, 8, 16 # debug
M, K, N = 4096, 6144, 2048 # real-world scale
device = Accelerator()
torch_A = torch.randn(M, K)
torch_B = torch.randn(K, N)
torch_result = (torch_A @ torch_B).numpy()
A = Tensor.from_numpy(torch_A.numpy()).to(device)
B = Tensor.from_numpy(torch_B.numpy()).to(device)

Build and test executing the CUDA graph for our MatMul kernel:

In [3]:
from max.graph import Graph, TensorType, DeviceRef, ops
def build_graph(session, algorithm):
    print('building cuda graph for', algorithm)
    with Graph("matmul_graph",
               input_types=[
                   TensorType(dtype=A.dtype, shape=A.shape, device=DeviceRef.from_device(device)),
                   TensorType(dtype=B.dtype, shape=B.shape, device=DeviceRef.from_device(device))
               ],
               custom_extensions=[Path(op_dir)]) as graph:
        A_value, B_value = graph.inputs
        output = ops.custom(
            name="my_matmul",
            device=DeviceRef.from_device(device),
            values=[A_value, B_value],
            out_types=[
                TensorType(dtype=A.dtype, shape=[
                        A_value.tensor.shape[0], B_value.tensor.shape[1]
                    ], device=DeviceRef.from_device(device))
            ],
            parameters={"algorithm": algorithm},
        )
        graph.output(output[0].tensor)
    print('loading cuda graph...')
    return session.load(graph) # compile the graph

from max.engine import InferenceSession
session = InferenceSession(devices=[device])
graph =  build_graph(session, "block_tiling")
mojo_result = graph.execute(A, B)[0].to_numpy()
print("test run:\n", mojo_result, end="\n\n")
print("reference:\n", torch_result)

building cuda graph for block_tiling
loading cuda graph...
test run:
 [[ 1.0453701e+02  3.8475451e+00 -1.4246112e+02 ...  4.2378632e+01
  -2.7167332e+01 -2.3036402e+01]
 [ 1.0757265e+02  1.0454322e+01  2.9893755e+01 ...  6.0303440e+00
  -2.8819391e+01 -4.9704781e+01]
 [ 1.0382000e+01 -2.4490438e+01 -2.9156441e+01 ... -4.6081947e+01
  -1.1421444e-01 -2.0481817e+01]
 ...
 [ 8.0971626e+01  4.6547596e+01  3.3553764e+01 ... -5.4983940e+00
  -3.2340786e+01 -5.2883773e+00]
 [ 6.6590858e+01 -9.7281067e+01  7.7483208e+01 ...  6.7729354e+00
  -4.2890152e+01  6.0306019e+01]
 [-1.3647166e+02 -2.8042696e+01 -8.2171486e+01 ... -6.3260212e+01
  -6.7414513e+00  5.1346752e+01]]

reference:
 [[ 1.0453696e+02  3.8475904e+00 -1.4246123e+02 ...  4.2378658e+01
  -2.7167330e+01 -2.3036337e+01]
 [ 1.0757260e+02  1.0454297e+01  2.9893637e+01 ...  6.0303860e+00
  -2.8819313e+01 -4.9704762e+01]
 [ 1.0382000e+01 -2.4490450e+01 -2.9156372e+01 ... -4.6081799e+01
  -1.1416578e-01 -2.0481800e+01]
 ...
 [ 8.0971672e+0

Verify kernel results:

In [4]:
assert numpy.allclose(mojo_result, torch_result, rtol=0, atol=0.005)
for row in range(torch_result.shape[0]):
    if numpy.allclose(torch_result[row], mojo_result[row], rtol=0, atol=0.005): continue
    print('mismatch row:', row)

The `tiled_register` kernel can be visualized using the [matmul_visualization.mojo](./matmul_visualization.mojo) and `matmul_visualization_gui/*` scripts.

Here is a screenshot:
![](./GEMM_visualization.png)

Run a complete benchmark for different algorithms:

In [6]:
import time
for algo in ["naive", "coalescing", "tiled", "tiled_register", "block_tiling"]:
    graph =  build_graph(session, algo)
    record = dict(torch=0, mojo=0)
    sampels = 5
    for _ in range(sampels):
        torch_A = torch.randn(M, K).to('cuda:0')
        torch_B = torch.randn(K, N).to('cuda:0')
        A = Tensor.from_numpy(torch_A.cpu().numpy()).to(device)
        B = Tensor.from_numpy(torch_B.cpu().numpy()).to(device)
        # torch
        torch.cuda.synchronize()
        begin = time.perf_counter()
        torch_result = torch_A @ torch_B
        torch.cuda.synchronize()
        record['torch'] += (time.perf_counter() - begin) / sampels
        # mojo
        torch.cuda.synchronize()
        begin = time.perf_counter()
        mojo_result = graph.execute(A, B)
        torch.cuda.synchronize()
        record['mojo'] += (time.perf_counter() - begin) / sampels
        assert numpy.allclose(mojo_result[0].to_numpy(), torch_result.cpu().numpy(), rtol=0, atol=0.005)
    print(algo, record)

building cuda graph for naive
loading cuda graph...
naive {'torch': 0.01311548782978207, 'mojo': 0.5172537408303469}
building cuda graph for coalescing
loading cuda graph...
coalescing {'torch': 0.013399246777407826, 'mojo': 0.13524032456334678}
building cuda graph for tiled
loading cuda graph...
tiled {'torch': 0.01450859212782234, 'mojo': 0.11470237262547014}
building cuda graph for tiled_register
loading cuda graph...
tiled_register {'torch': 0.01341570122167468, 'mojo': 0.2543464608024806}
building cuda graph for block_tiling
loading cuda graph...
block_tiling {'torch': 0.013868691981770098, 'mojo': 1.1725077461684124}


## Reference
[1] https://github.com/modular/modular/blob/main/examples/custom_ops/kernels/matrix_multiplication.mojo

[2] https://docs.modular.com/max/tutorials/custom-ops-matmul